# `lsdtopytools` a python binding to LSDTopoTools topographic analysis engine

This notebook presents briefly some base functionalities of `lsdtopytools`:
- how to load a DEM
- how to ready the tool for fluvial analysis
- how to calculate a basic river network
- how to calculate chi and $k_{sn}$
- how to visualise the results


## Importing the software and other utilities into the script

The fisrt step is to import the different packages we will utilise in this notebook. 
- `lsdtopytools` is the python binding to LSDTopoTools, it contains the specific algorithm for topographic analysis
- `matplotlib` for visualisation
- `numpy` to deal with array-type outputs (_e.g._ array-wide operations)
- `pandas` to deal with table-type data (_e.g._ river points)

In [2]:
import lsdtopytools as lsd
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [3]:
import lsdviztools.lsdbasemaptools as bmt
SB_DEM = bmt.ot_scraper(source = "SRTM30",longitude_W = -120.464655, longitude_E = -120.254214, latitude_S = 34.440538, latitude_N = 34.610770,prefix = "lg_conception")
SB_DEM.print_parameters()
SB_DEM.download_pythonic()
RasterFile = "lg_conception_SRTM30.tif"

The source is: SRTMGL1
The west longitude is: -120.464655
The east longitude is: -120.254214
The south latitude is: 34.61077
The north latitude is: 34.440538
The path is: ./
The prefix is: lg_conception
The resolution is: 30
I am going to download the following for you:
https://portal.opentopography.org/API/globaldem?demtype=SRTMGL1&south=34.440538&north=34.61077&west=-120.464655&east=-120.254214&outputFormat=GTiff
This might take a little while, depending on the size of the file. 
The filename will be:
./lg_conception_SRTMGL1.tif
The path and file without path are:
./  lg_conception_SRTMGL1.tif
Finished downloading


## Loading the DEM 

We use GDAL to load the DEM. Any common format is accepted (tif, bil, asc, ...) and the DEM should be in a metric projection (else, the units are not very instinctive). This function creates a LSDDEM python object. This object includes an underlying module called `lsdtt-xtensor-python` which deals with the communications with the `c++` code using `xtensor-python` from [Quantstack](https://github.com/QuantStack/).

The LSDDEM object is the base object of `lsdtopytools` to deal with DEM operations. It includes a number of options:

- `path (str)`: The relative or absolute path to your file. For example `C:Data/DEM/`.
- `file_name (str)`: The name of the file example: `dem.tif`.
- `already_preprocessed (bool)`: This is very important, preprocessed means that the DEM is ready for flow routines: the internal topographic pits have been filled or carved. By default, the DEM is considered as not preprocessed for flow routing which means it needs an extra step before being ready for fluvial analysis but you can set it as already preprocessed with this switch.
- `remove_seas (bool)`: Option to remove everything below a given sea level (defaulted at 0).
- `sea_level (float)`: Sea level utilised in case the sea are removed

In [4]:
mydem = lsd.LSDDEM(path = "./", file_name = "lg_conception_SRTMGL1.tif", already_preprocessed = False)

Loading the raster from file: ./lg_conception_SRTMGL1.tif
LOADING TOOK 0.05102872848510742
I am recasting your nodata values to -9999 (standard LSDTT)
PREPROC TOOK 0.0013248920440673828
Alright, let me summon control upon the c++ code ...
Got it.
INGESTINGINTO CPP TOOK 0.0038433074951171875
I am an empty LSDJunctionNetwork
Nrows:613 and ncols:758
out Nrows:613 and out ncols:758

I am an empty LSDJunctionNetwork
Yes, yes, we know. Ignore the above depressing debugging text, that's perfectly normal to feel empty sometimes.
FINALISATION TOOK 0.12454032897949219
lsdtopytools is now ready to roll!


## Preprocessing

The preprocessing makes sure that all the water in the DEM drains to the edge, i.e. water fluxes do not stop because of DEM noise. There are two main methods to preprocessed DEMs: 
- filling: filling algorithms (e.g. Barnes 2014) fills any area that has no connection to a DEM edge
- Carving: any pixels being local minima is connected to the nearest outlet with a trench

The preprocessing function has the following arguments:


In [5]:
help(mydem.PreProcessing)

Help on method PreProcessing in module lsdtopytools.lsdtopytools:

PreProcessing(filling=True, carving=True, minimum_slope_for_filling=0.0001) method of lsdtopytools.lsdtopytools.LSDDEM instance
    Description:
            Any dem is noisy at a certain extent. To process flow routines, this function proposes algorithm to preprocess dem cells and ensure there is no vicious pit blocking fow path.
            Filling is currently using Wang et al., 2006 and carving is using Lindsay et al., 2016.
            Filling makes sure that a minimum slope is induxed to each cells, carving breaches the pit to let the flow go.
    Arguments:
            filling (bool): do you want to fill?
            carving (bool): Wanna carve mate?
            minimum_slope_for_filling (float): Minimum gradient to induce between each cells when filling your dem.
    Return:
            Nothing, calculate the PPRaster in the cpp object
    Authors:
            Boris Gailleton
    Date:
            14/12/2018



In [6]:
mydem.CommonFlowRoutines()

WARNING!! You did not preprocessed the dem, I am defaulting it. Read the doc about preprocessing for more information.
Carving: implementation of Lindsay (2016) DOI: 10.1002/hyp.10648
Filling: implementation of Wang and Liu (2006): https://doi.org/10.1080/13658810500433453
Processing...
I am going to carve/breach your depressions in order to force flow paths.
DEM ready for flow routines!I am using an algorithm from Lindsay et al., 2016 DOI:https://doi.org/10.1002/hyp.10648
Implementation adapted from RichDEM: https://github.com/r-barnes/richdem
Not widely tested yet, It may break...

Processing common flow routines...
Done!


In [7]:
mydem.ExtractRiverNetwork( method = "area_threshold", area_threshold_min = 1500)

In [8]:
mydem.DefineCatchment( method="main_basin")

This appears to be a channel made up of a single pixel!


{'X': array([-120.42639], dtype=float32),
 'Y': array([34.55528], dtype=float32)}

In [9]:
mydem.GenerateChi(theta = 0.35, A_0 = 1)

In [10]:
mydem.ksn_MuddEtAl2014(target_nodes=30, n_iterations=60, skip=1, nthreads = 1)

I have generated ksn for the specified region!
Let me just save the result to the hdf5 file to keep track


'generated'

In [12]:
fig, ax = lsd.quickplot.get_basemap(mydem , figsize = (9,9), cmap = "gist_earth", hillshade = True, 
	alpha_hillshade = 1, cmin = None, cmax = None,
	hillshade_cmin = 0, hillshade_cmax = 1, colorbar = False, 
	fig = None, ax = None, colorbar_label = None, colorbar_ax = None, fontsize_ticks = 16, normalise_HS = True)
size_array = lsd.size_my_points(np.log10(mydem.df_ksn.drainage_area), 1,15)
cb = ax.scatter(mydem.df_ksn.x, mydem.df_ksn.y, lw=0, c= mydem.df_ksn.m_chi, cmap = "magma", zorder = 5, s=size_array)
plt.colorbar(cb)
plt.show()

Hillshading with altitude: 45, azimuth: 315 and z-factor: 1


In [14]:
%matplotlib inline
fig.show()